In [ ]:
import os
os.chdir('..')

In [ ]:
import re
import torch
import utils
import yaml
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from dataloader.data import TabularFeature, MIMICDataset, Normalizer
from dataloader.extractors import get_vocab
from dataloader.labels import get_labels
from dataloader.utils import BinnedEvent
from utils import prepare_batch, load_class, load_model, load_config

In [ ]:
DEVICE = 'cpu'
data_path = 'data/multitask'

In [ ]:
params = load_config('3aawlg9e')
params['patient_modelcls'], params['modelcls']

In [ ]:
params
params['min_word_count'] = 100000
params['batch_size'] = 4

In [ ]:
chart = TabularFeature('CHARTEVENTS', 'LABEL', 'VALUE', event_class=BinnedEvent, special_tok='🅒🅔', **params)
lab = TabularFeature('LABEVENTS', 'LABEL', 'VALUE', event_class=BinnedEvent, special_tok='🅛🅔', **params)
output = TabularFeature('OUTPUTEVENTS', 'LABEL', 'VALUE', event_class=BinnedEvent, special_tok='🅞🅔', **params)
dem = DemographicsFeature('dem', normalized_indices=[0, 1, 2])

tables = [chart, lab, output, dem]

labels = get_labels(DEVICE)

train_set = MIMICDataset(data_path, 'train', datalist_file='train_listfile.csv', mode='TRAIN',
                         tables=tables, labels=labels,
                         limit=None,
                         numericalize=False,
                         )

labels = get_labels(DEVICE, **params)
joint_vocab = get_vocab(**params)
tables = get_tables(vocab=joint_vocab,
                    load=True,
                    event_class=BinnedEvent,
                    **params)

In [ ]:
labels['phenotyping'].classes

In [ ]:
model = load_model(params, joint_vocab, tables, DEVICE)

In [ ]:
from functools import partial
from samplers import AgeSubjectRandomSampler
sampler = AgeSubjectRandomSampler(train_set)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=3,
                                           collate_fn=list,
                                           sampler=sampler,
                                           num_workers=0, pin_memory=True, drop_last=True)

In [ ]:
for sample in train_loader:
    break

# Positive Sample

In [ ]:
t = 0

In [ ]:
sorted(sample[0]['CHARTEVENTS'][t][1], key=lambda x: x[0])

# Next step

In [ ]:
sorted(sample[0]['CHARTEVENTS'][t+1][1], key=lambda x: x[0])

In [ ]:
sorted(sample[1]['CHARTEVENTS'][t+1][1], key=lambda x: x[0])

In [ ]:
sorted(sample[2]['CHARTEVENTS'][t+1][1], key=lambda x: x[0])